In [19]:
from bertopic import BERTopic
from datasets import load_dataset
import torch

In [26]:
pip install safetensors


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


In [2]:
dataset = load_dataset("mwarchalowski/grants", "no-shorts-no-dups")

In [3]:
from sentence_transformers import SentenceTransformer

In [4]:
embedding_model = SentenceTransformer("mixedbread-ai/mxbai-embed-large-v1")

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/171 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/111k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/677 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.24k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/297 [00:00<?, ?B/s]

In [7]:
splits = dataset["train"].train_test_split(0.1)
splits

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'abstr', 'text_len'],
        num_rows: 114100
    })
    test: Dataset({
        features: ['id', 'title', 'abstr', 'text_len'],
        num_rows: 12678
    })
})

In [16]:
embeddings = embedding_model.encode(splits["train"]["abstr"], show_progress_bar=True)
embeddings

Batches:   0%|          | 0/3566 [00:00<?, ?it/s]

array([[-0.2721274 ,  0.39993554,  0.04004147, ...,  0.5322828 ,
        -0.15304367,  0.42033535],
       [ 0.37619624,  0.5639731 , -0.2674801 , ...,  0.09050045,
        -0.24711074,  0.07018428],
       [ 0.07103545,  0.40288356,  0.30178797, ...,  0.11804905,
        -0.3014268 ,  0.22825383],
       ...,
       [-0.13096468,  0.5218836 ,  0.28385654, ...,  0.2532104 ,
        -0.7755157 , -0.25065824],
       [ 0.03244508,  0.8145087 , -0.19566627, ..., -0.11640655,
        -0.32859585,  0.19593976],
       [ 0.39871967, -0.47737214,  1.6064315 , ...,  0.4581581 ,
        -0.25867087, -0.68136466]], dtype=float32)

In [17]:
embeddings.shape

(114100, 1024)

In [18]:
type(embeddings)

numpy.ndarray

In [21]:
torch_embeddings = torch.from_numpy(embeddings)

In [22]:
torch_embeddings.shape

torch.Size([114100, 1024])

In [25]:
torch.save(torch_embeddings, "embeddings.pt")